In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
header = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}
url = 'https://women.#######.net/players'
page = requests.get(url, headers=header)
page

<Response [200]>

In [3]:
##### Web scrapper for infinite scrolling page #####
driver = uc.Chrome()
driver.get('https://women.volleybox.net/players')
time.sleep(5)  # Allow 2 seconds for the web page to open
#scroll_pause_time = 4 
scroll_height = driver.execute_script("return document.body.scrollHeight;")
scroll_height_list = []
while True:
    # scroll one screen height each time
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")  

    
    last_scroll_height = driver.execute_script("return document.body.scrollHeight;")
    scroll_height = last_scroll_height
    scroll_height_list.append(scroll_height)
    try:
        wait = WebDriverWait(driver, 2)
        wait.until(EC.presence_of_element_located((By.ID, "items_count_hid")))
    except TimeoutException:
        break


In [ ]:
players_list_all = driver.find_elements(By.ID, "player_list_container")
players_list = players_list_all[0].find_elements(By.CSS_SELECTOR, "[href]")
links = [elem.get_attribute('href') for elem in players_list]
len(links)

In [6]:
driver.quit()

In [64]:
volleyball_players_dataset = pd.DataFrame(columns=['NAME', 'RANKING', 'CURRENT_RANKING', 'NATIONALITY', 'POSITION', 'BIRTHDATE',
                                                  'HEIGHT', 'WEIGHT', 'SPIKE', 'BLOCK', 'DOMINANT_HAND',
                                                  'JOINED_TEAM', 'TRANSFER_DATE_JOINED', 'TRANSFER_DATE_LEFT',
                                                  'TEAMMATES', 'CLUBS', 'AWARDS', 'TOURNAMENTS', 'MATCHES'])

In [18]:
def get_team_name(team_text, i):
    try:
        return team_text.find_all('div', class_='div-1-1')[i].find('a').text.strip()
    except:
        return None
def get_joined_date(joined_date_text, i):
    try:
        return joined_date_text.find_all('div', class_='div-1-1')[i].find('div').text[1:-1].split('-')[0].strip()
    except:
        return None
def get_left_date(joined_left_text, i):
    try:
        return joined_left_text.find_all('div', class_='div-1-1')[i].find('div').text[1:-1].split('-')[1].strip()
    except:
        return None

In [67]:
def get_player_data(data_labels, data_values, ranking=None, current_ranking=None, nationality=None,
                    position=None, birthdate=None, height=None, weight=None, spike=None, block=None, dominant_hand=None):
    for label, value in zip(data_labels, data_values):
        if 'Ranking' == label.text:
            ranking = value.text
        elif 'Ranking in' == label.text[:10]:
            current_ranking = value.text
        elif 'Nationality' in label.text:
            nationality = value.text
        elif 'Position' in label.text:
            position = value.text
        elif 'Birthdate' in label.text:
            birthdate = value.text
        elif 'Height' in label.text:
            height = value.text
        elif 'Weight' in label.text:
            weight = value.text
        elif 'Spike' in label.text:
            spike = value.text
        elif 'Block' in label.text:
            block = value.text
        elif 'Dominant hand' in label.text:
            dominant_hand = value.text
        else: continue
    
    return ranking, current_ranking, nationality, position, birthdate, height, weight, spike, block, dominant_hand

In [ ]:
for link in tqdm(links): #player_lists[0].find_all('a'):
    player_page = requests.get(link, headers=header) #.get('href')
    player_soup = BeautifulSoup(player_page.content, "html.parser")
    #print(player_soup.find_all('div', class_='profile_header_text')[0].find('h1').text[1:-1])
    #print(player_soup.find_all('div', class_='div-1-1')[0].find('a').text.strip())
    #print(player_soup.find_all('div', class_='div-1-1')[0].find('div').text[1:-1].split('-')[0].strip())
    #print(player_soup.find_all('div', class_='div-1-1')[0].find('div').text[1:-1].split('-')[1].strip())
    
    # player_data
    player_data_labels = player_soup.find_all('div', class_='new_box pRelative')[0].find_all('dt', class_='info-header tar marginBottom10')
    player_data_values = player_soup.find_all('div', class_='new_box pRelative')[0].find_all('dd', class_='info-data marginBottom10')
    ranking, current_ranking, nationality, position, birthdate, height, weight, spike, block, dominant_hand = get_player_data(player_data_labels, player_data_values)
        
    # team_data
    team_name = [None]
    if len(player_soup.find_all('div', class_='new_box player-experience')) != 0:
        team_data = player_soup.find_all('div', class_='new_box player-experience')[0]
        team_name = []
        joined_date = []
        left_date = []
        for i in range(len(team_data)):
            if get_team_name(team_data, i) != None:
                team_name.append(get_team_name(team_data, i))
            if get_joined_date(team_data, i) != None:
                joined_date.append(get_joined_date(team_data, i))
            if get_left_date(team_data, i) != None:
                left_date.append(get_left_date(team_data, i))
    
    # player_screen_data
    club, tournament, match, award, teammate = '0', '0', '0', '0', '0'
    player_wide_tab = player_soup.find_all('div', class_='profile-wide-tabs-inner')[0].find_all('span')
    for i in player_wide_tab:
        if 'Clubs' in i:
            club = i.find('var').text
        elif 'Tournaments' in i:
            tournament = i.find('var').text
        elif 'Matches' in i:
            match = i.find('var').text
        elif 'Awards' in i:
            award = i.find('var').text
        elif 'Teammates' in i:
            teammate = i.find('var').text
        else: continue
        
    # create_df
    row_dict = {'NAME' : [player_soup.find_all('div', class_='profile_header_text')[0].find('h1').text[1:-1]]*len(team_name),
                'RANKING' : [ranking]*len(team_name),
                'CURRENT_RANKING' : [current_ranking]*len(team_name),
                'NATIONALITY' : [nationality]*len(team_name),
                'POSITION' : [position]*len(team_name),
                'BIRTHDATE' : [birthdate]*len(team_name),
                'HEIGHT' : [height]*len(team_name),
                'WEIGHT' : [weight]*len(team_name),
                'SPIKE' : [spike]*len(team_name),
                'BLOCK' : [block]*len(team_name),
                'DOMINANT_HAND' : [dominant_hand]*len(team_name),
                'JOINED_TEAM' : team_name,
                'TRANSFER_DATE_JOINED' : joined_date,
                'TRANSFER_DATE_LEFT' : left_date,
                'TEAMMATES' : [teammate]*len(team_name),
                'CLUBS' : [club]*len(team_name),
                'AWARDS' : [award]*len(team_name),
                'TOURNAMENTS' : [tournament]*len(team_name),
                'MATCHES' : [match]*len(team_name)
                }
    
    # add_df
    volleyball_players_dataset = pd.concat([volleyball_players_dataset, pd.DataFrame.from_dict(row_dict, orient='index')],
                                           ignore_index=True)
    


In [ ]:
links[1672]

In [ ]:
row_dict

In [55]:
volleyball_players_dataset.tail()

,NAME,RANKING,CURRENT_RANKING,NATIONALITY,POSITION,BIRTHDATE,HEIGHT,WEIGHT,SPIKE,BLOCK,DOMINANT_HAND,JOINED_TEAM,TRANSFER_DATE_JOINED,TRANSFER_DATE_LEFT,TEAMMATES,CLUBS,AWARDS,TOURNAMENTS,MATCHES
198,Federica Belli,None,None,Italy,Outside Hitter,None,None,None,None,None,None,3epc Asp Civitavecchia,2023/24,2023/24,17,1,0,1,0
199,Lana Rnjak,None,None,Serbia,Setter,2007-11-07,176cm,None,None,None,None,GŽOK Srem 2,2023/24,2023/24,13,1,0,0,0
200,Kanako Hayashi,None,None,Japan,Libero,None,None,None,None,None,None,Polonia SideOut London,2023/24,2023/24,10,1,0,1,0
201,Ayaka Watanabe,None,1755,Japan,Outside Hitter,None,168cm,None,None,None,Right,Ryukoku University,2023/24,still,8,2,0,5,25
202,Ayaka Watanabe,None,1755,Japan,Outside Hitter,None,168cm,None,None,None,Right,Kyoto Tachibana High School,2020/21,2022/23,8,2,0,5,25
